In [1]:
import pandas as pd
import numpy as np
import operator

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('quake_db_1965-2016.csv')
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,Place
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic,Unknown
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic,"Ngidihopitonu, ID"
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic,"Pangai, TO"
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic,Unknown
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic,"Dapdap, PH"


In [3]:
country_code = 'IN'

In [4]:
country_df = df[df['Place'].str.contains(country_code)]
country_df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,Place
58,02/18/1965,04:26:37,25.011,94.186,Earthquake,55.0,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM860307,ISCGEM,ISCGEM,ISCGEM,Automatic,"State of Manipur, IN"
402,04/04/1966,06:42:13,12.010,92.483,Earthquake,10.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM848084,ISCGEM,ISCGEM,ISCGEM,Automatic,Union Territory of Andaman and Nicobar Islands...
468,06/27/1966,10:59:19,29.706,80.935,Earthquake,25.0,NaN,NaN,6.3,MW,...,NaN,NaN,NaN,NaN,ISCGEM846871,ISCGEM,ISCGEM,ISCGEM,Automatic,"Dharchula, IN"
494,08/15/1966,02:15:32,28.565,78.961,Earthquake,27.0,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM844491,ISCGEM,ISCGEM,ISCGEM,Automatic,"Shahabad, IN"
528,09/26/1966,05:10:57,27.419,92.576,Earthquake,14.4,NaN,NaN,5.9,MW,...,NaN,NaN,NaN,NaN,ISCGEM843878,ISCGEM,ISCGEM,ISCGEM,Automatic,"Bomdila, IN"


In [5]:
country_df = country_df[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Depth', 'Place']]
_, _, country_df['Year'] = country_df['Date'].str.split('/').str
country_df.loc[:, 'Year'] = country_df.loc[:, 'Year'].astype(int)

In [6]:
country_df.head()

,Date,Latitude,Longitude,Magnitude,Depth,Place,Year
58,02/18/1965,25.011,94.186,5.6,55.0,"State of Manipur, IN",1965
402,04/04/1966,12.010,92.483,5.8,10.0,Union Territory of Andaman and Nicobar Islands...,1966
468,06/27/1966,29.706,80.935,6.3,25.0,"Dharchula, IN",1966
494,08/15/1966,28.565,78.961,5.6,27.0,"Shahabad, IN",1966
528,09/26/1966,27.419,92.576,5.9,14.4,"Bomdila, IN",1966


In [7]:
year_list = country_df['Year'].to_list()
year_dict = {}
for year in year_list:
    year_dict[year] = year_list.count(year)
print(year_dict)

{1965: 1, 1966: 4, 1967: 4, 1969: 1, 1970: 1, 1971: 2, 1973: 2, 1974: 2, 1977: 1, 1978: 2, 1979: 1, 1980: 2, 1981: 1, 1982: 1, 1984: 2, 1986: 2, 1987: 1, 1988: 1, 1989: 2, 1991: 2, 1993: 1, 1995: 1, 1997: 3, 1999: 2, 2000: 1, 2001: 4, 2002: 1, 2003: 4, 2004: 9, 2005: 5, 2006: 4, 2007: 1, 2008: 7, 2009: 1, 2010: 1, 2011: 1, 2012: 1, 2013: 2, 2014: 1, 2016: 1}


In [8]:
year_value = max(year_dict.items(), key=operator.itemgetter(1))[0]
year_value

2004

In [9]:
year_marks = np.linspace(1965, 2016, 15)
year_marks = [int(i) for i in year_marks]
year_marks

[1965,
 1968,
 1972,
 1975,
 1979,
 1983,
 1986,
 1990,
 1994,
 1997,
 2001,
 2005,
 2008,
 2012,
 2016]

In [10]:
external_scripts = ['https://www.google-analytics.com/analytics.js']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__,
    external_scripts=external_scripts,
    external_stylesheets=external_stylesheets)
app.config['suppress_callback_exceptions'] = True

In [11]:
app.layout = html.Div([
    html.Div(id='map-update'),
    html.Div([
        dcc.Slider(
            id='year-slider',
            min=min(year_list),
            max=max(year_list),
            step=1,
            marks={i : '{}'.format(i) for i in year_marks},
            value=year_value
        )
    ], className='container')
])

In [12]:
def getDataYearValue(value):
    data = country_df[country_df['Year'] == value]
    return data

def PlotDensityMap(lat, lon, z, radius, colorscale):
    density_map_trace = go.Densitymapbox(
        lat=lat,
        lon=lon,
        z=z,
        radius=radius,
        colorscale=colorscale,
    )
    return density_map_trace

def LayoutDensity(height, width, mapbox_style, c_lat, c_lon, zoom):
    layout_density_map = go.Layout(
        height=height,
        width=width,
        autosize=True,
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox_style=mapbox_style,
        mapbox_center_lat=c_lat,
        mapbox_center_lon=c_lon,
        mapbox=dict(
            zoom=zoom
        )
    )
    return layout_density_map

In [13]:
@app.callback(
    Output('map-update', 'children'), [Input('year-slider', 'value')]
)
def output_update(value):
    country_df = getDataYearValue(value)
    center_lat = country_df[country_df['Magnitude'] == country_df['Magnitude'].max()]['Latitude'].to_list()[0]
    center_lon = country_df[country_df['Magnitude'] == country_df['Magnitude'].max()]['Longitude'].to_list()[0]
    mags = country_df['Magnitude'].to_list()
    lats = country_df['Latitude'].to_list()
    lons = country_df['Longitude'].to_list()
    map_trace = PlotDensityMap(lats, lons, mags, 10, 'Electric')
    layout_map = LayoutDensity(600, 980, 'stamen-terrain', center_lat, center_lon, 3)
    visualization = html.Div([
        dcc.Graph(
            id='density-map',
            figure={'data' : [map_trace], 'layout' : layout_map}
        ),
    ])
    return visualization    

In [14]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.0&m=1568744085 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1566936214 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.0&m=1568744085 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.2.1&m=1565030553 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.0&m=1568744085 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:25] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.1.0&m=1568744095 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/sameer/.local/lib/python3.7/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Oct/2019 15:05:36] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Oct/2019 15:05:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:39] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/sameer/.local/lib/python3.7/site-packages/dash/dash.py", line 1

...

127.0.0.1 - - [19/Oct/2019 15:05:43] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Oct/2019 15:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:49] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/sameer/.local/lib/python3.7/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Oct/2019 15:05:50] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Oct/2019 15:05:51] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sameer/.local/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/sameer/.local/lib/python3.7/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Oct/2019 15:05:52] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Oct/2019 15:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 15:05:56] "POST /_dash-update-component HTTP/1.1" 200 -
